In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## detecting and cropping faces

In [3]:
done = True
done = False

In [4]:
import os
import glob
datapath_faces = 'dataset_faces'
datapath = 'dataset'

In [5]:
from gaze import init, Data
args = init(batch_size=8, no_cuda=True, verbose=1)
d = Data(args)

Found 801 sample images;  641  to train 160 to test


In [6]:
if True:
    n_show = 10
    for target in d.classes:
        for filename in glob.glob(os.path.join(datapath, target) + '/*.png')[-n_show:]:
            print(filename)

dataset/blink/2018-09-04_Laurent_236.png
dataset/blink/2018-09-05_Laurent_125.png
dataset/blink/2018-09-14_Laurent_242.png
dataset/blink/2018-09-05_Laurent_084.png
dataset/blink/2018-09-05_Laurent_090.png
dataset/blink/2018-09-14_Laurent_121.png
dataset/blink/2018-09-04_Laurent_180.png
dataset/blink/2018-10-03_Laurent_116.png
dataset/blink/2018-09-05_Laurent_085.png
dataset/blink/2018-09-14_Laurent_069.png
dataset/center/2018-09-04_Laurent_194.png
dataset/center/2018-09-04_Laurent_157.png
dataset/center/2018-09-05_Laurent_244.png
dataset/center/2018-09-05_Laurent_091.png
dataset/center/2018-09-04_Laurent_16.png
dataset/center/2018-09-14_Laurent_243.png
dataset/center/2018-09-04_Laurent_221.png
dataset/center/2018-09-04_Laurent_235.png
dataset/center/2018-10-03_Laurent_089.png
dataset/center/2018-10-03_Laurent_076.png
dataset/left/2018-09-04_Laurent_181.png
dataset/left/2018-09-14_Laurent_134.png
dataset/left/2018-09-14_Laurent_108.png
dataset/left/2018-09-05_Laurent_091.png
dataset/lef

In [7]:
import imageio
if not done:
    import time
    from gaze import FaceExtractor
    FE = FaceExtractor()
    timings = []
    for target in d.classes:
        for filename in glob.glob(os.path.join(datapath, target, '*.png')):
            frame = imageio.imread(filename)
            t0 = time.time()
            face = FE.face_extractor(frame)
            t, b, l, r = FE.get_bbox(frame)
            t1 = time.time()
            timings.append(t1-t0)
            if False:
                # Create figure and axes
                fig, ax = plt.subplots(figsize=(15, 8))

                # Display the image
                ax.imshow(frame)

                # Create a Rectangle patch
                from matplotlib import patches
                rect = patches.Rectangle((t, l), r-l, t-b, linewidth=1, edgecolor='r', facecolor='none')

                # Add the patch to the Axes
                ax.add_patch(rect)
                plt.show()    
            if False:
                # Create figure and axes
                fig, ax = plt.subplots(figsize=(15, 8))

                # Display the cropped image
                ax.imshow(face)

                plt.show()                

            filename_face = filename.replace(datapath, datapath_faces)
            imageio.imwrite(filename_face, face) 


ValueError: Could not load bitmap "/Users/laurentperrinet/research/CatchTheEye/dataset/left/2018-09-14_Laurent_108.png": No known reason.

In [ ]:
if not done:
    timings = np.array(timings) * 1000
    print('timings in ms =', timings.mean(), '+/-', timings.std()) 

In [ ]:
if not done:
    fig, ax = plt.subplots(figsize=((8, 5)))
    n, bins, patches = ax.hist(timings, bins=np.linspace(100, 200, 100), alpha=.4)
    ax.vlines(np.median(timings), 0, n.max(), 'g', linestyles='dashed', label='median = %.3f ms' % np.median(timings))
    #ax.vlines(25, 0, n.max(), 'r', linestyles='dashed', label='chance level')
    #ax.vlines(100, 0, n.max(), 'k', label='max')
    ax.set_xlabel('Timings (ms)')
    ax.set_ylabel('Smarts')
    ax.legend(loc='best')
    if True:
        plt.show() 
    else:
        plt.savefig('figures/dlib_timings.png')

## training the model on the cropped faces

In [ ]:
from gaze import init
#epochs = 400
#args = init(verbose=0, epochs=epochs)
args = init(verbose=1)
print('Parameters=', args)

path = '_Regard.pt'

In [ ]:
from gaze import init
epochs = 400
args = init(verbose=0, epochs=epochs)
path = '_Regard_400.pt'

In [ ]:
!ls -l {path}
#!rm {path}

In [ ]:
from gaze import ML
ml = ML(args)
ml.train(path=path)

## testing the model

In [ ]:
Accuracy = ml.test()
print('Accuracy={:.1f}%'.format(Accuracy*100))

Showing the one which are wrong:

## showing the images which are wrongly classified

In [ ]:
import time
from gaze import FaceExtractor
FE = FaceExtractor()
timings = []
for target in d.classes:
    for filename in glob.glob(os.path.join(datapath, target) + '/*.png'):
        frame = imageio.imread(filename)
        img_face = FE.face_extractor(frame)
        pred = ml.classify(img_face, ml.dataset.test_transform)
        pred_label = ml.dataset.dataset.classes[pred.argmax()]
        if not pred_label == target:
            print('For ', filename, ', Pred =', pred_label, 'True =', target, ', P=', ['%.3f' % p for p in pred])
            plt.imshow(img_face)
            plt.show()

In [ ]:
!rm  dataset/center/2018-09-05_Laurent_073.png

# showing the trained weights

https://github.com/utkuozbulak/pytorch-cnn-visualizations/blob/master/README.md


In [ ]:
child = ml.model.children()

convolayer = [i for i in ml.model.children()][1]
convolayer

weights = convolayer.weight.data.clone()
weights.shape

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from torchvision import utils

def plotkernel(tensor, figname, ch=0, allkernels=False, nrow=3, padding=1):
    n, c, w, h = tensor.shape
    if allkernels: tensor = tensor.view(n*c,-1,w,h )
    elif c != 3: tensor = tensor[:,ch,:,:].unsqueeze(dim=1)
        
    rows = np.min( (tensor.shape[0]//nrow + 1, 64 )  )    
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, padding=padding)
    fig, ax = plt.subplots( figsize=(nrow,rows) )
    ax.imshow(grid.numpy().transpose((1, 2, 0)))
    plt.setp(ax, xticks=[], yticks=[])

    plt.savefig(figname)

for i in range(2):
    convolayer = [i for i in ml.model.children()][i]
    weights = convolayer.weight.data.clone()
    plotkernel(weights, figname = 'kernel_layer' + str(i) + '.png')

    